In [5]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import time
import datetime
# import gurobipy as gp  # import the installed package

In [ ]:
'''
Instance Parameters
===============================
Customer Location: I
Facility Location: J
Resource Type: K
Competitor Facility Location: L

U_LT : Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
U_T : Max allocation limit for resource k, Dim : (k)
U_L : Max capacity for facility j, Dim : (j)
V : Attractiveness yield of resource k allocated to facility j, Dim : (j, k)
H : Max demand for customer i, Dim : (i)
D : Distance between customer i and facility j , Dim : (i, j)
D_comp : Distance between customer i and competitor l, Dim : (i, l)
F : Fixed cost for building facility j, Dim : (j)
C : Cost per unit of extra attractiveness at facility j, Dim : (j)
B : Cost of allocating a unit of resource k to facility j, Dim : (j, k)
===============================
'''
config = {
    'i_amount' :5, #五個客戶
    'j_amount' :3, #三個設廠地點
    'k_amount' :2, #兩種可分配資源（貨車, 小車)
    'l_amount' :3,
    'M': 1000000,
    'lambda_G': 0.5,
    'A_opponent_bar': [3,6],
}

I = set(range(1, config['i_amount']+1))  
J = set(range(1, config['j_amount']+1))  
K = set(range(1, config['k_amount']+1))  
L = set(range(1, config['l_amount']+1))

U_LT = {}  
U_T = {}  
U_L = {}  
V = {}   
H = {}  
D = {}  
D_comp = {}  
F = {}  
C = {}  
B = {} 
  
def G(tai):
    return 1 - np.exp(-config['lambda_G'] * tai)

In [ ]:
# Initialize the model
model = Model("Competitive Facility Location")


'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
===============================
'''
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13


# P is sum of j for each i, so dim here is (i)
P = [quicksum((quicksum(V[j, k] * x[j, k] for k in K) + aX[j]) / (D[i, j]**2) for j in J) for i in I] # 3.8
TA = [ P[i] + quicksum(config['A_opponent_bar'][l] / (D_comp[i, l]**2) for l in L) for i in I ] # 3.7

# Constraints
# Resource allocation constraints
model.addConstrs((x[j, k] <= U_LT[j, k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= U_T[k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= U_L[j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

# Objective function: Maximize profit by attracting customers - costs
objective = quicksum(H[i] * G(TA[i]) * (P[i]/TA[i]) for i in I) - quicksum(F[j] * y[j] + C[j] * aX[j] + quicksum(B[j, k] * x[j, k] for k in K) for j in J)
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")


In [ ]:
# for var in cfl.getVars():
#     print(f"{var.varName} = {var.x}"
# print("")